In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36'}

In [ ]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    #options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)


In [ ]:
def open_popup():
    element=driver.find_element_by_xpath('//*[@id="megaHead2015"]/div[1]/div[2]/a[3]')
    
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    
    driver.find_element_by_xpath('//*[@id="megaGnb"]/div[1]/div[2]/a[1]').click()

In [ ]:
def change_frame():
    iframe = driver.find_element_by_xpath('//*[@id="ifrm_quick_search"]')
    driver.switch_to.frame(iframe)
    

In [ ]:
def get_onclick(xpath_value):
    get_onclick = []
    for i in driver.find_elements_by_xpath(xpath_value):
        get_onclick.append(i.get_attribute('onclick'))
        
    return(get_onclick)

In [ ]:
def switch_to_tch():
    driver.find_element_by_xpath('//*[@id="menu_quick02"]').click()

In [ ]:
def get_tch_home():
    
    tch_home =[]
    for i in range(2,10):
        value = '//*[@id="imgTec01_%d"]' %i

        driver.execute_script(driver.find_element_by_xpath(value).get_attribute('onclick'))

        time.sleep(0.5)

        tch_onclick=get_onclick('/html/body/div/div[9]/form/table/tbody/tr/td[3]/div[2]/table/tbody/tr/td[1]/table/tbody/tr[4]/td/div/table/tbody/tr[*]/td[*]/a')


        for i in tch_onclick:
            driver.execute_script(i)

            time.sleep(0.5)

            tch_home.extend(get_onclick('//*[@id="layer_quick_teacher02"]/span[2]/a'))
        
    return(tch_home)



In [ ]:
def get_page_url(tch_home):
    page_url=[]
    for i in tch_home:
        page_url.append('https://www.megastudy.net%s' %re.split("'",i)[1])
    return(page_url)

In [ ]:
def select_lec():
    word_check=[]
    for i in driver.find_elements_by_xpath('//*[@id="tecChrTabArea"]/ul/li[*]'):
        word_check.append(i.text)

    num=word_check.index('단과강좌')+1

    driver.find_element_by_xpath('//*[@id="tecChrTabArea"]/ul/li[%s]' %num).click()

    time.sleep(0.5)

In [ ]:
def get_data_1():
    name = []
    title=[]
    link=[]
    if len(driver.find_elements_by_xpath('//*[@id="divChrTabArea"]/ul/li[*]'))==0:

        for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[1]/a'):
            name.append(i.text)    

        for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[2]/div/p[3]/a'):
            title.append(i.text)
            link.append(i.get_attribute('href'))

        subject = driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[1]/h2/a[1]/span').text
        subject = [subject]*len(title)
    else :
        for j in driver.find_elements_by_xpath('//*[@id="divChrTabArea"]/ul/li[*]'):
            driver.execute_script(j.find_element_by_tag_name('a').get_attribute('onclick'))

            time.sleep(0.5)

            for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[1]/a'):
                name.append(i.text)    

            for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[2]/div/p[3]/a'):
                title.append(i.text)
                link.append(i.get_attribute('href'))

            subject = driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[1]/h2/a[1]/span').text
            subject = [subject]*len(title)
            
    return(name, title, link)

        

In [ ]:
def get_data_2(link):
    data_2=[]
    num= 0
    for i in link:
        print(i)
        num=num+1
        print(num,'/',len(link))
        
        table=pd.read_html(i)
        for t in table:
            if list(t)==['강의명','맛보기']:
                sample=t.iloc[:,0]
                break
        list1=list(range(1,len(sample)+1))
        list2= list(sample)
        
        try:
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])
        except:
            index_value= list(pd.isnull(list2)).index(True)
            list2.pop(index_value)
            list2.pop(index_value)
            
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])
    
    return(data_2)

In [ ]:
def get_sub(lec_link):
    sub=[]
    num=0
    for i in lec_link:
        num=num+1
        
        print(num, '/', len(lec_link))
        
        res=requests.get(i,headers=header)
        soup=BeautifulSoup(res.content, 'html.parser',from_encoding='cp949')
        value=soup.find('p',{'class':'lstedu_bookinfo--teacher'}).find('strong').a.text
        sub.append(re.sub('[^가-힣]','',re.split(']', value)[0]))

    return(sub)

In [ ]:
def get_id(lec_link):
    lec_id=[]
    tch_id=[]
    for i in lec_link:
        lec_id.append(re.split('&',re.split('=',lec_link[0])[1])[0])
        tch_id.append(re.split('=',lec_link[0])[3])
    return(lec_id,tch_id)

In [ ]:
def get_complete(lec_link):
    complete=[]
    for i in lec_link:
        if re.split('&',re.split('=',i)[2])[0]=='1':
            complete.append('O')
        else:
            complete.append('X')
    return(complete)

In [ ]:
driver = open_chromedriver()

In [ ]:
driver.get('https://www.megastudy.net/')

In [ ]:
open_popup()

In [ ]:
change_frame()

In [ ]:
switch_to_tch()

In [ ]:
tch_home=get_tch_home()

In [ ]:
page_url=get_page_url(tch_home)

In [ ]:
### 기존 프레임으로 전환
driver.switch_to.parent_frame()

In [ ]:
tch_name =[]
title = []
lec_link = []
num = 0
for i in page_url:
    num =num+1
    driver.get(i)
    
    time.sleep(1)
    
    try:
        select_lec()
    
        [x.extend(y) for x,y in zip([tch_name, title, lec_link], get_data_1())]
    except:
        pass
    
    print(num,'/',len(page_url))

In [ ]:
sub = get_sub_tch_id(lec_link)

In [ ]:
data_2 = get_data_2(lec_link)

In [ ]:
lec_id,tch_id=get_id(lec_link)

In [ ]:
complete=get_complete(lec_link)

In [ ]:
data = pd.DataFrame({'사이트':['메가스터디']*len(sub),'과목':sub,'선생님':tch_name,'선생님ID':tch_id,'강좌명':title,'강좌ID':lec_id,'완강여부':complete,'강의번호&챕터명&시간':data_2,'링크':lec_link})